In [2]:
import os
import numpy as np
import keras
from skimage.io import imread
from skimage.transform import resize
from matplotlib import pyplot as plt
from random import seed, sample
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
from pprint import pprint
from sklearn.metrics import accuracy_score
from random import randint
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
# image = imread(os.path.join(self.folder_data, couple[0] + '.png'), as_grey=True)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [0]:
!tar -xvf EnglishHnd.tgz

In [20]:
rootDir = './English/Hnd/Img'
n_images = 6420
images = np.zeros((n_images,) + (20, 15))

labels = []

j = 0
for dirName, subdirList, fileList in os.walk(rootDir):
    i = 0
    if 'Sample' in dirName:
        label = int(dirName[-2:])
    # if label < 11 or label > 36:
    #     continue
        if label > 36:
            label -= 26
    for fname in fileList:
        if 'png' not in fname:
            continue
        full_path = os.path.join(dirName, fname)
        images[j] = resize(imread(full_path, as_grey=True), (20, 15)) 
        labels.append(label)
        j += 1
        i += 1
        # if i == 5:
        #     break
images = images[:len(labels)]

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [21]:
print(len(labels))
print(len(images))

3410
3410


In [0]:
k = sample(range(len(images)), len(images))
im_shuf = images[k]
labels_shuf = np.array(labels)[k]
    
ocr = {
    'images': im_shuf,
    'data': im_shuf.reshape((im_shuf.shape[0], -1)), # / 255.0
    'target': labels_shuf
}

In [0]:
total = len(ocr['target'])
population = range(total)
k = int(np.floor(total * 0.1))
test = sample(population, k)
train = [i for i in population if i not in test]
images_train = ocr['images'][train]
data_train = ocr['data'][train]
labels_train = ocr['target'][train]
images_test = ocr['images'][test]
data_test = ocr['data'][test]
labels_test = ocr['target'][test]

In [0]:
model = LinearSVC()
param_grid = {'C':  list(np.arange(0.1,1.5,0.1))}

In [10]:
gs = GridSearchCV(model, param_grid, n_jobs=-1, cv=3, verbose=4)
gs.fit(data_train, labels_train)
pprint(sorted(gs.grid_scores_, key=lambda x: -x.mean_validation_score))

Fitting 3 folds for each of 14 candidates, totalling 42 fits
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] .................................. C=0.1, score=0.245357 -  29.1s
[CV] C=0.1 ...........................................................
[CV] .................................. C=0.1, score=0.222222 -  29.4s
[CV] C=0.2 ...........................................................
[CV] .................................. C=0.1, score=0.241345 -  29.6s
[CV] C=0.2 ...........................................................
[CV] .................................. C=0.2, score=0.224155 -  32.3s
[CV] C=0.2 ...........................................................
[CV] .................................. C=0.2, score=0.239492 -  33.1s
[CV] C=0.30000000000000004 ...........................................
[CV] .................................. C=0.2, score=0.235410 -  32.9s
[CV] C=0.3000000

[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  5.9min


[CV] .................................. C=0.8, score=0.195169 -  33.1s
[CV] C=0.8 ...........................................................
[CV] .................................. C=0.8, score=0.214076 -  33.2s
[CV] C=0.9 ...........................................................
[CV] .................................. C=0.8, score=0.201780 -  33.5s
[CV] C=0.9 ...........................................................
[CV] .................................. C=0.9, score=0.203865 -  33.1s
[CV] C=0.9 ...........................................................
[CV] .................................. C=0.9, score=0.207234 -  33.6s
[CV] C=1.0 ...........................................................
[CV] .................................. C=0.9, score=0.207715 -  35.4s
[CV] C=1.0 ...........................................................
[CV] .................................. C=1.0, score=0.203865 -  35.2s
[CV] C=1.0 ...........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed: 11.7min finished


[mean: 0.23623, std: 0.01009, params: {'C': 0.1},
 mean: 0.23297, std: 0.00649, params: {'C': 0.2},
 mean: 0.22744, std: 0.00720, params: {'C': 0.30000000000000004},
 mean: 0.22450, std: 0.00645, params: {'C': 0.4},
 mean: 0.21668, std: 0.00886, params: {'C': 0.5},
 mean: 0.21408, std: 0.00490, params: {'C': 0.6},
 mean: 0.21277, std: 0.00565, params: {'C': 0.7000000000000001},
 mean: 0.20951, std: 0.01066, params: {'C': 1.0},
 mean: 0.20626, std: 0.00171, params: {'C': 0.9},
 mean: 0.20365, std: 0.00783, params: {'C': 0.8},
 mean: 0.19941, std: 0.00255, params: {'C': 1.3000000000000003},
 mean: 0.19583, std: 0.00488, params: {'C': 1.4000000000000001},
 mean: 0.19225, std: 0.00945, params: {'C': 1.2000000000000002},
 mean: 0.19094, std: 0.01116, params: {'C': 1.1}]


In [11]:
y_pred = gs.predict(data_test)
print ('Test set shape: ', data_test.shape)
print ('Target shape: ', labels_test.shape)
print ('Accuracy on train set: ', accuracy_score(labels_train, gs.predict(data_train)))
print ('Accuracy on test set: ', accuracy_score(labels_test, y_pred))

Test set shape:  (341, 400)
Target shape:  (341,)
Accuracy on train set:  0.5529488432714239
Accuracy on test set:  0.28152492668621704


In [32]:
x_train, x_test, y_train, y_test = train_test_split(im_shuf, labels_shuf, random_state=2, train_size=0.8)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [33]:
num_classes = 37
batch_size = 128

X_train = x_train.reshape(x_train.shape[0], 20, 15 , 1).astype('float32')
X_test = x_test.reshape(x_test.shape[0], 20, 15 , 1).astype('float32')
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(20, 15, 1)))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.summary()
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size,
                        nb_epoch=40,
                        verbose=1)

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/40
2728/2728 [==============================] - 7s 2ms/step - loss: 3.5944 - acc: 0.0323
Epoch 2/40
2728/2728 [==============================] - 6s 2ms/step - loss: 3.5640 - acc: 0.0418
Epoch 3/40
2728/2728 [==============================] - 6s 2ms/step - loss: 3.4448 - acc: 0.0729
Epoch 4/40
2728/2728 [==============================] - 6s 2ms/step - loss: 3.2575 - acc: 0.1202
Epoch 5/40
2728/2728 [==============================] - 6s 2ms/step - loss: 3.0238 - acc: 0.1657
Epoch 6/40
2728/2728 [==============================] - 6s 2ms/step - loss: 2.7862 - acc: 0.2229
Epoch 7/40
2728/2728 [==============================] - 6s 2ms/step - loss: 2.5946 - acc: 0.2573
Epoch 8/40
2728/2728 [==============================] - 6s 2ms/step - loss: 2.3647 - acc: 0.3130
Epoch 9/40
2728/2728 [==============================] - 6s 2ms/step - loss: 2.1085 - acc: 0.3897
Epoch 10/40
2728/2728 [==============================] - 6s 2ms/step - loss: 2.0285 - acc: 0.3977
Epoch 11/40
2728/2728 [======

In [34]:
scores = model.evaluate(X_test, y_test, verbose = 10 )
print ( scores )

[1.4876500301696687, 0.6217008797653959]


In [0]:
ynew = model.predict_classes(X_test)

In [39]:
y_test[:10], ynew[:10]

(array([36, 19,  4, 32,  2, 24, 20, 18, 26, 17]),
 array([ 4, 19, 29, 32, 22, 34, 20, 18, 26, 17]))